<center><font size = "10"> Week 10 - Network Stimulation<center>
<center><font size = "8">Home Work <center>

# Ex1. Regular synchronous stimulation. Half weight connection.

<font size='3'>Load the five morphology reconstructions assigned to your group in week 2, using the class "Cell" defined in these tutorials. Connect the cells with weight = 1, as it is done in tutorial 01. Add exponential synapses (weight 1, tau = 2 ms) to a random section of the basal dendrite of each cell. Create a regular synchronous stimulation and apply it to each exponential synapse. Run a simulation of 300 ms and show the plots for the voltage and the spikes.

<font size='3'>1. Reduce the weigth connection between cells to 0.5, plot the same result as before and compare with the previous simulation. What do you see now? Explain.
    
<font size='3'>2. Reduce the weight connection of the expenential synapses to 0.5, plot the result and compare it with the first simulation (weight connection between cells = 1). What do you see? Explain [2 pts]

# Ex2. Irregular asynchrounous stimulation. Inhibitory cells effect = negative weight. 

<font size='3'>Load the five morphology reconstructions assigned to your group in week 2, using the class "Cell" defined in these tutorials. Connect the cells with the following weights: w(Pyr - Pyr) = 1, w(Pyr - Int) = 1, w(Inh - Pyr) = -0.01, w(Int - Int) = -0.01. Add exponential synapses (weight 1, tau = 2 ms) to a random section of the basal dendrite of each cell. Create a irregular asynchronous stimulation, as it is done in tutorial 02 and apply it to each exponential synapse. Run a simulation that last at least to contain the last stimulation. Plot the results in two plots one showing the cells voltage response and other with the spike detection. Could you describe what do you see? What is the firing dynamics of these small network?
    
<font size='3'>Change the connection weight between Int - Pyr and to - 1 and compare with the previous simulation. Describe what is happening now. [1 pt] 

# Ex3. Remove inhibitory cells

<font size='3'> Perform the same simulation as before, but this time remove the inhibitory cells from the network. Compare this result with the previous simulations. What is the role of the inhibitory cells in this small network? [1 pt]

# Ex4. Poissonian stimulation

<font size='3'>Load the five morphology reconstructions assigned to your group in week 2, using the class "Cell" defined in these tutorials. Connect the cells with the following weights: w(Pyr - Pyr) = 1, w(Pyr - Int) = 1, w(Inh - Pyr) = -0.01, w(Int - Int) = -0.01. Add exponential synapses (weight 1, tau = 2 ms) to a random section of the basal dendrite of each cell. Using the function (numpy.ramdom.poisson()) create a poissonian stimulation process different for each exponential synapse with no more than 10 stimulations. Run a simulation that last at least to contain the last stimulation. Plot the voltage response of each cell and the spike detection.
    
    
<font size='3'>Describe what do you see. What do you think a poissonian stimulation is useful for when simulationg networks? Is there any difference bewteen a poissonian stimulation and an irregular asynchronous stimualtion? [2 pts]

<font size='3'><font color='red'>__IMPORTANT!!__ The code from the tutorials generates random connections between cells, if you want to compare results you have to keep the same connections (same axon section connected to same basal section) between the cells. The same happens with the location of the exponential synapses.